In [7]:
# workspace = "dev"
# apiVersion = "5.2"
# moduleName = "Ed-Fi"
# schoolYear = None
# districtId = None

StatementMeta(spark3p3sm, 0, 8, Finished, Available)

In [8]:
%run OEA_py

StatementMeta(, 0, -1, Finished, Available)

2023-06-23 16:55:23,984 - OEA - INFO - Now using workspace: dev
2023-06-23 16:55:23,985 - OEA - INFO - OEA initialized.


In [9]:
oea.set_workspace(workspace)

StatementMeta(spark3p3sm, 0, 10, Finished, Available)

2023-06-23 16:55:24,599 - OEA - INFO - Now using workspace: dev


In [10]:
districtPath = districtId if districtId != None else "All"
schoolYearPath = schoolYear if schoolYear != None else "All"
items = oea.get_folders(f'stage1/Transactional/{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/ed-fi')
print(items)

StatementMeta(spark3p3sm, 0, 11, Finished, Available)

['absenceEventCategoryDescriptors', 'academicHonorCategoryDescriptors', 'academicSubjectDescriptors', 'academicWeeks', 'accommodationDescriptors', 'accountClassificationDescriptors', 'accountCodes', 'accountabilityRatings', 'accounts', 'achievementCategoryDescriptors', 'actuals', 'additionalCreditTypeDescriptors', 'addressTypeDescriptors', 'administrationEnvironmentDescriptors', 'administrativeFundingControlDescriptors', 'ancestryEthnicOriginDescriptors', 'assessmentCategoryDescriptors', 'assessmentIdentificationSystemDescriptors', 'assessmentItemCategoryDescriptors', 'assessmentItemResultDescriptors', 'assessmentItems', 'assessmentPeriodDescriptors', 'assessmentReportingMethodDescriptors', 'assessmentScoreRangeLearningStandards', 'assessments', 'attemptStatusDescriptors', 'attendanceEventCategoryDescriptors', 'behaviorDescriptors', 'bellSchedules', 'budgets', 'calendarDates', 'calendarEventDescriptors', 'calendarTypeDescriptors', 'calendars', 'careerPathwayDescriptors', 'charterApprov

In [11]:
def should_ingest(entity_path):
    raw_path = f'stage1/Transactional/{entity_path}'
    if not oea.path_exists(raw_path):
        return False
    if len(oea.get_folders(raw_path)) > 0:
        batch_type, source_data_format = oea.get_batch_info(raw_path)
        logger.info(f'Ingesting from: {raw_path}, batch type of: {batch_type}, source data format of: {source_data_format}')
        source_url = oea.to_url(f'{raw_path}/{batch_type}_batch_data')

        if batch_type == 'delta' or batch_type == 'additive': source_url = f'{source_url}/{oea.get_latest_folder(source_url)}' 

        return oea.get_folder_size(source_url) > 0
    else:
        return False

def ingest_edfi_dataset(tables_source):
    full_source_path = f"stage1/Transactional/{tables_source}"
    items = oea.get_folders(full_source_path)
    options = {'header':False}
    for item in items: 
        table_path = full_source_path +'/'+ item
        try:
            entity_path = f"{tables_source}/{item}"
            print(entity_path)
            if item == 'metadata.csv':
                logger.info('ignore metadata processing, since this is not a table to be ingested')
            else:
                if(should_ingest(entity_path)):
                    oea.ingest(entity_path, 'id', options)
        except AnalysisException as e:
            # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
            pass

StatementMeta(spark3p3sm, 0, 12, Finished, Available)

In [12]:
ingest_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/ed-fi')
ingest_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/tpdm')

StatementMeta(spark3p3sm, 0, 13, Finished, Available)

Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors
2023-06-23 16:55:28,223 - OEA - INFO - Ingesting from: stage1/Transactional/Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors, batch type of: delta, source data format of: json
2023-06-23 16:55:28,438 - OEA - INFO - Ingesting from: stage1/Transactional/Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors, batch type of: delta, source data format of: json
source_path is: abfss://oea@stoeadablickw3.dfs.core.windows.net/dev/stage1/Transactional/Ed-Fi/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors/delta_batch_data
2023-06-23 16:55:48,567 - py4j.java_gateway - INFO - Callback Server Starting
2023-06-23 16:55:48,571 - py4j.java_gateway - INFO - Socket listening on ('127.0.0.1', 34815)
2023-06-23 16:55:52,324 - py4j.java_gateway - INFO - Callback Connection ready to receive messages
2023-06-23 16:55:52,353 - py4j.java_gateway - INFO - Rece

ParseException: 
Possibly unquoted identifier ldb_dev_s2i_ed-fi detected. Please consider quoting it with back-quotes as `ldb_dev_s2i_ed-fi`(line 1, pos 44)

== SQL ==
CREATE DATABASE IF NOT EXISTS ldb_dev_s2i_ed-fi
--------------------------------------------^^^
